# MU5EEH15: Interactive Robot Learning

Objective: Learn how to do programming in `Python` interactive robot learning.
- Machine learning / Human Robot Interaction (HRI)
- Reinforcement learning (rewards, human feedback)
- Supervised learning
- Immitation learning

**Organization**: Lectures and Practical Labs **(TP 40%)** + Final project **(40%)** and exam **(20%)**

**Lecturer**: Silvia TULLI - mail: silvia.tulli@sorbonne-universite.fr

**Student**: William WU - mail: william.wu@etu.sorbonne-universite.fr

___

# Agenda 29/09/2025

Recap Questionnaire on Human-Interactive Robot Learning
- Fundamentals of RL Course
- Practice with Jupyter Notebooks
- Final Questionnaire on today's material, both course and practice

# Learning Goals

By the end of this lecture, you should be able to:
- Frame a sequential decision making problem as an MDP
- Explain the concept of value function (V) and action function (Q)
- Apply value iteration and policy iteration to solve an MDP
- Explain general RL concepts such as Model-based vs. Model-free RL and the exploration-exploitation tradeoff
- Master the following algorithms :Value Iteration, Policy Iteration, Dynamic Programming, Q-learning, SARSA.

___

# Table of Contents

1. [Sequential Decision Making - Simple Gridworld](#sequential-decision-making---simple-gridworld)
    - [Markov Decision Process](#markov-decision-process)
    - [Different Policies](#different-policies)
2. [Machine Learning Paradigms](#machine-learning-paradigms)
    - [Reinforcement Learning Framework](#reinforcement-learning-framework)
    - [Sequential Decision Problems (MDPs/POMDPs)](#sequential-decision-problems-mdpspomdps)
    - [Basic Decision-Making Problem (Deterministic)](#basic-decision-making-problem-deterministic) 
    - [Principle of Optimality (Bellman's Principle)](#principle-of-optimality-bellmans-principle)
    - [Rewards](#rewards)
    - [Exploration/Exploitation Tradeoff](#explorationexploitation-tradeoff)
    - [Terminology](#terminology)
    - [Offline vs Online Planning](#offline-vs-online-planning)
    - [Solving MDPs](#solving-mdps)
    - [Q-learning vs Deep Q-learning](#q-learning-vs-deep-q-learning)

# Sequential Decision Making - Simple Gridworld

- Robot/Mouse (agent)
- Grab the cheese (goal)
- Maze (environment)
- Cells of the maze (possible states)
- Arrows (agent’s possible actions)

Noisy environment - if you move forward you do not necessarily go forward

After the learning happens and after we realise how to solve it optimally that would be a potential trajectory when everything is done

## Markov Decision Process

The "Markov" in "Markov decision process" refers to the underlying structure of state transitions that still follow the Markov property. MDP is a model for sequential decision making. The Markov property means that evolution of the Markov process in the future depends only on the present state and does not depend on past history. 

## Different Policies

Policies are the way for programs to aim their objectives.
- Supervised policy is an algorithm updated with human wants that orientates the policy
- Greedy policy is maximizing the reward.
- Epsilon-greedy policy is a policy that introduces epsilon which sets the level of exploring the program will do

___




# Machine Learning Paradigms

- Supervised learning: labeled data, with training on labels and then do predictions.
- Unsupervised learning: no label data, feature extractions (no ground truths) then classification.
- Reinforcement learning: chasing for rewards (different algorithms)

## Reinforcement Learning Framework

<img src="RL_diagram.png" width="500" height="500">

When we do reinforcement learning:
- There is no supervisor, only a reward signal (e.g., this was good/bad, this gives you 10 points)
- Feedback may be delayed, but can also be instantaneous depending on the environment
- Time really matters: we talk about sequential processes (non i.i.d data)
- The agent is influenced by the sequence of data it receives

Reinforcement learning is the science of decision-making → try to find the optimal way to make decisions A number of impressive successes in the last decade.

___



## Sequential Decision Problems (MDPs/POMDPs)

- **Goal**: select actions to maximise total future reward
- Actions may have **long term consequences**
- **Reward** may be delayed 
- It may be better to **sacrifice** immediate reward to gain more long-term reward
- Actions change the environment state
- Actions may have rewards that appear many steps later
- The **sequence of actions matters**, not just individual choices
- Hard to know which past actions led to current rewards

___



## Basic decision-making problem (deterministic)

- **Aim:** Find the best sequence of control actions to minimize some cost while satisfying system dynamics and constraints
- Discrete-time model with additive cost (central assumption)

#### System: $ x_{k+1} = f_k(x_k, u_k), \quad k = 0, \ldots, N $

- $x_{k+1}$ is the condition: state vector at time step k
- $u_k$ is the action: control input at time step k
- $f_k(x_k, u_k)$ is the state transition function (how the system evolves from one time step to the next)
- The next state depends on the current state and the control we apply
- No randomness; given state and control, next state is known

#### Control constraints: $ u_k \in U(x_k) $

- This represents physical limitations (e.g., maximum motor torque, speed limits)
- The constraints can be state-dependent, meaning available actions may change based on where you are

#### Cost: $ J(x_0; u_0, \ldots, u_{N-1}) = g_N(x_N) + \sum_{k=0}^{N-1} g_k(x_k, u_k) $

- $g_N(x_N)$ is the terminal cost (penalty for where we end up)
- $g_k(x_k, u_k)$ is the stage cost at each time step (running costs)
- The cost combines: immediate costs at each step + final cost

#### Decision-Making Problem: $ J^*(x_0) = \min_{u_k \in U(x_k), k=0, \ldots, N-1} J(x_0; u_0, \ldots, u_{N-1}) $

- Optimal value function (minimum achievable cost from initial state $x_0$)
- We're finding the control sequence that minimizes total cost. 
- Subject to: system dynamics + control constraints

___

## Principle of optimality (Bellman's Principle)

Let $\{u_0^*,u_1^*,...u_{N-1}^*\}$ be an optimal control sequence, which together with $x_0^*$ determines the corresponding state sequence $\{x_0^*,x_1^*,...x_N^*\}$. Consider the subproblem whereby we are at $x_k^*$ at time $k$. 

We wish to minimize the cost-to-go from time $k$ to time $N$, i. e.,

$ g_k(x_k^*,u_k) + \sum_{m=k+1}^{N-1} g_m(x_m,u_m) + g_N(x_N) $

Then the truncated optimal sequence $\{u_0^*,u_1^*,...u_{N-1}^*\}$ is optimal for the subproblem. 

Tail of optimal sequences optimal for tail subproblems

#### Applying the principle of optimality

- need only to compare the concatenations of immediate decisions and optimal decisions 
    - significant decrease in computation / possibilities
- in practice: carry out this procedure backward in time

#### Dynamic Programming Algorithm

Start with $ J_N^*(x_N) = g_N(x_N) $, for all $ x_N $

and for $ k = N - 1, \ldots, 0 $, let

$
J_k^*(x_k) = \min_{u_k \in U(x_k)} \left[ g(x_k, u_k) + J_{k+1}^* (f(x_k, u_k)) \right] \quad ,\forall x_k
$

Once the functions $ J_0^*, \ldots, J_N^* $ have been determined, the optimal sequence can be determined with a forward pass.

Bellman's principle: the optimal cost from state $x_k$ equals the minimum over all feasible controls of [immediate cost + optimal future cost].

After computing backwards all value functions you construct the actual optimal trajectory

- DP guarantees finding the globally optimal solution (not just locally optimal) due to the principle of optimality
- The algorithm must be computed for all possible states $x_k$, which can be computationally challenging in high-dimensional problems.
- Instead of computing the values over the entire state space via DP, we can be smarter by only computing the values around our best-guess trajectory and iteratively closing in on the actual optimal solution.

___

## Rewards

- First step is understanding the reward signal
- Scalar feedback $R_t$ about how well the agent is doing at time step $t$
- The goal of the agent is to maximise the cumulative reward
- Reinforcement learning is based on the reward hypothesis.

All goals can be expressed by the maximisation of expected cumulative rewards

## Exploration/exploitation tradeoff

To figure out what is the meaning of the scalar value the agent’s has to explore Exploration/exploitation tradeoff
- Should the agent explores what it knows or looks for new solutions?
- How fast should you decrease your exploration rate?
- Epsilon-greedy: taking the best action most of the time and a random action from time to time

## Terminology

- A learning agent has to:
    - Sense the state of its environment
    - Take actions that affect the state
    - Have a goal or goals relating to the state of the environment

- A Markov Decision Process (MDP) is a mathematical formalisation for modeling decision making and include these aspects:
    - Sensation, Action, Goal

- Policy - maps from perceived states of the environment to actions to be taken in those states (e.g., in psychology stimulus-response rules or associations)
- Reward signal - defines the goal in the problem. Single number that tells what are the good and bad events for the agent (e.g., in biological systems pleasure/pain)
- Value Function - specifies what is good in the long run
- Model of the environment - mimics the behavior of the environment

___

## Offline vs Online Planning

- Offline planning (MDPs)
    - MDP is given
    - The agent find the optimal policy for the MDP
    - The agent acts in the environment

- Online Planning (RL)
    - Learning is required
    - The agent has access to a set of available actions and information about the state it is in.


## Solving MDPs

Techniques for solving MDPs (and POMDPs) can be separated into three categories:
- **Value-based techniques** aim to _learn the value of states_ (or learn an estimate for value of states) and actions: that is, they learn value functions or Q functions. We then use _policy extraction_ to get a policy for deciding actions.
- **Policy-based techniques** _learn a policy directly_, which _completely by-passes learning values of states or actions all together_. It's important if, for example, the state space or the action space are massive or infinite. If the action space is infinite, then using policy extraction is not possible because we must iterate over all actions to find the optimal one. If we
learn the policy directly, we do not need this.
- **Hybrid techniques** that combine _value and policy-based techniques_.

## Q-learning vs Deep Q-learning

<img src="value_iteration.png" width="800" height="500">
<img src="policy_iteration.png" width="800" height="500">


*she went speed-running for the slides, gotta read now...*
